In [1]:
import sqlite3
import csv

# Correspondance nom scientifique - nom commun

In [2]:
# Tableau de correspondance Nom latin, Nom commun :
nomlatin_csv = 'data/nomlatin_lesarbresfr.csv'

with open(nomlatin_csv, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    
    nomslatins = {}
    for row in reader:
        nomvernaculaire = row[0]
            
        # Remarque: il y a parfois plusieur nom latin proposé
        all_latinname = [ n.strip() for n in  row[1].split(',') ]

        for latinname in all_latinname:
            nomslatins[latinname] = nomvernaculaire


print(len(nomslatins))

556


In [3]:
# Lecture des triplets ( genre, espece, variete  ) :

db_filename = 'data/arbres.db'
db = sqlite3.connect(db_filename)


cursor = db.cursor()

cursor.execute("""
SELECT count(*) AS c, genre_bota, espece, variete FROM arbres
GROUP BY genre_bota, espece, variete
ORDER BY genre_bota
             """)

#nametuples = cursor.fetchmany(10)
nametuples = cursor.fetchall()

In [4]:
import unidecode

In [5]:
def filterlist( text, liste, startswith=False ):
    """ retourne une nouvelle liste ne comprenant que les élements (string)
        dans lesquels est présent text (string)
        
        startswith=True : force 'text' à être present au début
    """
    
    if not text: return []
    
    # case insensitive
    text = text.lower()
    liste = [ m.lower() for m in liste ]
    # no accent
    text = unidecode.unidecode( text )
    liste = [ unidecode.unidecode(m) for m in liste ]
    
    # specials rules  (by hand)
    if text == 'thuja': text = 'thuya'
    if text == 'sambuccus': text = 'sambucus'
    if text == 'sambuccus': text = 'sambucus'
    if text == 'cladastris': text = 'cladrastis'
    if text == 'eleagnus': text = 'elaeagnus'
    if text == 'pistacia': text = 'pistachia'
         
    # starts with  or  inside ?
    if startswith:
        crible = lambda A, B: B.startswith( A )
    else:
        crible = lambda A, B: A in B

    
    filtered_list = [ ele for ele in liste if crible( text, ele ) ]
    return filtered_list

In [6]:
def keep_only_commonpart( liste ):
    """ ne garde que les lettres présentent dans tous les mots de la 'liste', en partant début
    """
    
    N = min( [len(mot) for mot in liste] )

    common = []
    for i in range(N):
        if len( { mot[i] for mot in liste } ) == 1:
            common.append( liste[0][i] )
        else:
            break

    common = ''.join( common ).strip(' ')

    return common

In [7]:
def findmatch( triplet ):
    """ pour un triplet (genre, espece, variete) cherche le nom scientifique de la liste
        qui correspond le mieux
        retourne Faux si non trouvé
        retourne la partie commune si plusieurs possibilités
    """
    
    genre, espece, variete = triplet
        
    nomspossibles = list( nomslatins.keys() )
    
    # filter
  
    nomspossibles_G = filterlist( genre, nomspossibles, startswith=True )
        
    if len(nomspossibles_G) == 1 :
        return nomspossibles_G[0]
    elif len(nomspossibles_G) == 0 :
        return False
    else:
        
        nomspossibles_GE = filterlist( espece, nomspossibles_G )
        
        if len(nomspossibles_GE) ==  1:
            return nomspossibles_GE[0]
        elif len(nomspossibles_GE) == 0 :
            match = keep_only_commonpart( nomspossibles_G )
            if match : match += '*'
            return match
        
        else:
            
            nomspossibles_GEV = filterlist( variete, nomspossibles_GE )
            
            if len(nomspossibles_GEV) ==  1:
                return nomspossibles_GEV[0]
            elif len(nomspossibles_GEV) == 0 :
                match = keep_only_commonpart( nomspossibles_GE )
                if match : match += '*'
                return match
            else:
                match = keep_only_commonpart( nomspossibles_GEV )
                if match : match += '*'
                return match
            

In [8]:
identified = []
non_identified = []
for row in nametuples:
   
    match = findmatch( row[1:] )
    if match:
        identified.append(( row, match ))
    else:
        non_identified.append( row )
        
print( len(identified), 'triplet trouvés' )
print( len(non_identified), 'non trouvés' )
print( len(nametuples), 'au total' )

525 triplet trouvés
22 non trouvés
547 au total


In [9]:
# trié par nombre d'arbres :
sorted( non_identified, key=lambda x:x[0], reverse=True )

[(783, None, None, None),
 (77, 'Tetradium', 'danielli', None),
 (22, 'Trachycarpus', 'fortunei', None),
 (9, 'Pteroceltis', 'tatarinowii', None),
 (7, 'Tetradium', 'danielli', 'Hupehensis'),
 (6, None, 'kobus', None),
 (6, 'Pterostyrax', 'hispida', None),
 (4, 'Chitalpa', 'tashkentensis', None),
 (3, None, 'fragmantissima', None),
 (3, 'Clerodendron', 'tricotonum', None),
 (3, 'Euodia', 'danielli', None),
 (3, 'Fontanesia', 'phillyreoides', 'Fortunei'),
 (2, 'Clerodendron', None, 'Fargesii'),
 (2, 'Hovenia', 'dulcis', None),
 (2, 'Pteroceltis', 'tatarinowii', 'Maxim'),
 (1, 'Fontanesia', 'phillyreoides', None),
 (1, 'Nerprun', 'alaternus', None),
 (1, 'Philodendron', 'erubescens', None),
 (1, 'Poncinos', None, None),
 (1, 'Pteroceltis', None, 'Maxim'),
 (1, 'Sorbopyrus', 'auricularis', None),
 (1, 'Tetradium', 'tomentosa', None)]

In [10]:
# -: the end :-
db.close()